In [32]:
import pandas as pd
from haversine import haversine
import datetime
import gc
import os
import json
today_str=str(datetime.datetime.now().date())


In [33]:
writer_folder="/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/Q3_2018/SOTF_20180619/"+today_str+"/"
try:
    os.stat(writer_folder)
except:
    os.mkdir(writer_folder)

In [34]:
store_list_Q3=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/Q3_2018/SOTF_20180619/SOTF_48_20180619.csv",dtype=str)

In [35]:
store_level_zips=pd.read_csv("/home/jian/Projects/Big_Lots/New_TA/zips_in_new_ta/sales_by_zip (Store level).csv",dtype=str)
store_level_zips_Q3_SOTF=store_level_zips[store_level_zips['location_id'].isin(store_list_Q3['location_id'])]
store_level_zips_Q3_SOTF=pd.merge(store_level_zips_Q3_SOTF,store_list_Q3,on="location_id",how="left")

store_level_zips_Q3_SOTF_P=store_level_zips_Q3_SOTF[store_level_zips_Q3_SOTF['revenue_flag']=="P"]
store_level_zips_Q3_SOTF_P_keep_store=store_level_zips_Q3_SOTF_P[['zip','location_id','revenue_flag','Market']]
store_level_zips_Q3_SOTF_P=store_level_zips_Q3_SOTF_P[['zip','revenue_flag','Market']].drop_duplicates()

store_level_zips_Q3_SOTF_S=store_level_zips_Q3_SOTF[store_level_zips_Q3_SOTF['revenue_flag']=="S"]
store_level_zips_Q3_SOTF_S=store_level_zips_Q3_SOTF_S[~store_level_zips_Q3_SOTF_S['zip'].isin(store_level_zips_Q3_SOTF_P['zip'])]
store_level_zips_Q3_SOTF_S.shape


(879, 7)

In [36]:
store_zip=pd.read_excel("/home/jian/Projects/Big_Lots/New_TA/P_S_Selections/Store List 6.14.18 225PM.xlsx",sheetname="Store List",dtype=str)
store_zip=store_zip.iloc[:1415,:]
store_zip=store_zip[['Store','Zip']].rename(columns={"Store":"location_id","Zip":"store_zip"})
store_zip=store_zip[store_zip['location_id'].isin(store_list_Q3['location_id'])]
store_zip.shape

(48, 2)

In [37]:
store_level_zips_Q3_SOTF_S=pd.merge(store_level_zips_Q3_SOTF_S,store_zip,on="location_id",how="left")
store_level_zips_Q3_SOTF_S['zip_1st_dig']=store_level_zips_Q3_SOTF_S['zip'].apply(lambda x:x[0])
store_level_zips_Q3_SOTF_S['store_1st_dig']=store_level_zips_Q3_SOTF_S['store_zip'].apply(lambda x:x[0])
store_level_zips_Q3_SOTF_S=store_level_zips_Q3_SOTF_S[store_level_zips_Q3_SOTF_S['zip_1st_dig']==store_level_zips_Q3_SOTF_S['store_1st_dig']]
store_level_zips_Q3_SOTF_S.shape

(724, 10)

In [38]:
store_level_zips_Q3_SOTF_S_keep_store=store_level_zips_Q3_SOTF_S[['zip','location_id','revenue_flag','Market']]
store_level_zips_Q3_SOTF_S=store_level_zips_Q3_SOTF_S[['zip','revenue_flag','Market']].drop_duplicates()
store_level_zips_Q3_SOTF_S=store_level_zips_Q3_SOTF_S[~store_level_zips_Q3_SOTF_S['zip'].isin(store_level_zips_Q3_SOTF_P['zip'])]

agg_zip_by_Market=store_level_zips_Q3_SOTF_P.append(store_level_zips_Q3_SOTF_S)
agg_zip=agg_zip_by_Market[['zip','revenue_flag']].drop_duplicates()
agg_zip_by_store=store_level_zips_Q3_SOTF_P_keep_store.append(store_level_zips_Q3_SOTF_S_keep_store)

In [39]:
agg_zip_by_store=agg_zip_by_store[agg_zip_by_store['revenue_flag']=="P"]
agg_zip_by_Market=agg_zip_by_Market[agg_zip_by_Market['revenue_flag']=="P"]
agg_zip=agg_zip[agg_zip['revenue_flag']=="P"]

In [40]:
new_store_list=[x for x in store_list_Q3['location_id'] if x not in store_level_zips_Q3_SOTF['location_id'].unique()]
store_list_20171115=pd.read_table("/home/jian/Projects/Big_Lots/New_TA/MediaStormStoreList_Nov15.txt",sep="|",dtype=str)
zip_centers=json.load(open("/home/jian/Docs/Geo_mapping/center_of_rentrak_zip.json","r"))
result=pd.DataFrame()
i=0
for store in new_store_list:
   
    lat=float(store_list_20171115['latitude_meas'][store_list_20171115['location_id']==store].tolist()[0])
    long=float(store_list_20171115['longitude_meas'][store_list_20171115['location_id']==store].tolist()[0])
    zip_for_store=store_list_20171115['zip_cd'][store_list_20171115['location_id']==store].tolist()[0]
    market=store_list_Q3['Market'][store_list_Q3['location_id']==store].tolist()[0]
    
    store_center=(lat,long)
    
    result_df=pd.DataFrame()

    for zip_5 in zip_centers.keys():    
        dist=haversine(store_center,zip_centers[zip_5],miles=True)
        if dist<=10:        
            df=pd.DataFrame({"zip_cd":zip_5,"Center_Lat":zip_centers[zip_5][0],"Center_Long":zip_centers[zip_5][1],"Distance_Mile":dist},index=[i])
            i=i+1
            result_df=result_df.append(df)
    result_df['Store_id']=store
    result_df['Store_zip']=zip_for_store
    result_df=result_df[['Store_id','Store_zip','zip_cd','Center_Lat','Center_Long','Distance_Mile']]
    result_df['Market']=market
    result=result.append(result_df)
new_mile_10_store=result[['zip_cd','Store_id','Market']].rename(columns={"zip_cd":"zip","Store_id":"location_id"})
new_mile_10_store['revenue_flag']="Within_10_miles"
new_mile_10_store=new_mile_10_store[['zip','location_id','revenue_flag','Market']]
new_mile_10=result[['zip_cd','Store_id','Market']].rename(columns={"zip_cd":"zip","Store_id":"revenue_flag"})
new_mile_10['revenue_flag']=new_mile_10['revenue_flag'].apply(lambda x: "10_miles_for_store_"+x)



In [41]:
agg_zip_by_Market=agg_zip_by_Market.append(new_mile_10)
del new_mile_10['Market']
agg_zip=agg_zip.append(new_mile_10)
agg_zip_by_store=agg_zip_by_store.append(new_mile_10_store)

In [42]:
writer=pd.ExcelWriter(writer_folder+"BL_P_only_zips_for_SOTF_48_stores"+today_str+".xlsx",engine="xlsxwriter")
agg_zip.to_excel(writer,"deduplicated_zips",index=False)
agg_zip_by_Market.to_excel(writer,"zips_by_Market",index=False)
agg_zip_by_store.to_excel(writer,"zips_by_store",index=False)
writer.save()
